In [22]:
# Note to import from .py files, must follow structure
# from <.py filename excluding '.py'> import <class name>
# Optionslam creds: aspringfastlaner Options2018

# Importing necessary models
import smtplib
import pandas as pd
import numpy as np
import datetime as dt
import pandas.stats.moments as st
from pandas import ExcelWriter
import matplotlib.pyplot as plt
import os
import seaborn as sns
import matplotlib.dates as dates
# import matplotlib.ticker as ticker
from lxml import html
import requests
import webbrowser
from bs4 import BeautifulSoup as bs
import json
import csv
import sched, time
import pandas_datareader as datareader
from pandas_datareader.data import Options
from alpha_vantage.timeseries import TimeSeries
import matplotlib.pyplot as plt

# Alpha Vantage API Key
# 5HZEUI5AFJB06BUK

# ts = TimeSeries(key='5HZEUI5AFJB06BUK', output_format='pandas')
# data, meta_data = ts.get_intraday(symbol='MSFT',interval='1min', outputsize='full')
# data['close'].plot()
# plt.title('Intraday Times Series for the MSFT stock (1 min)')
# For intraday
# https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol=MSFT&interval=1min&apikey=d5HZEUI5AFJB06BUK&datatype=csv

# For daily
# https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol=MSFT&apikey=5HZEUI5AFJB06BUK&datatype=csv

In [2]:
'''
Pulling S&P 500 Names
'''

def pull_sp500_list():
    site = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
    res = requests.get(site)
    soup = bs(res.text, 'lxml')
    table = soup.find_all('table')[0]

    tickers = []
    names = []
    gics = []

    # Looping through the soup lxml text table format
    # and splitting each row as a individual string
    # and parsing string to retrieve the date,
    # open, and close information.
    i = 1
    for row in table.find_all('tr'):
        if i == 1:
            i += 1
            continue
        # Individual row stores current row item and delimits on '\n'
        individual_row = str(row).split('\n')
        # row_items is parsed string for each current row where each
        ticker = individual_row[1].split('">')[-1].split('<')[0]
        tickers.append(ticker)
        name = individual_row[2].split('">')[-1].split('<')[0]
        names.append(name)
        gic = individual_row[4].split('>')[1].split('<')[0]
        gics.append(gic)

    sp500 = pd.DataFrame({'Name': names, 'GIC': gics}, index = tickers)
    return sp500

In [25]:
'''
Function for pulling implied volatility from option slam for single ticker
'''

def optionslam_scrape(ticker):
    site = 'https://www.optionslam.com/earnings/stocks/' + ticker
    res = requests.get(site)
    soup = bs(requests.get(site).text, "lxml")
    soup = soup.prettify()
    earnings_dict = {'Ticker': ticker}
    
    # Check if there's weekly options
    curr7_implied = "Current 7 Day Implied Movement:"
    implied_move_weekly = "Implied Move Weekly:"
    nextearnings = "Next Earnings Date:"
    if curr7_implied not in soup:
        return 'No Weeklies'
    
    # Parsing if weekly options exist
    # Next earnings date and before or after
    earnings_start_string = "Next Earnings Date:"
    earnings_end_string = '</font>'
    raw_earnings_string = (soup.split(earnings_start_string))[1].split(earnings_end_string)[0].replace('\n','').strip()
    
    try:
        earnings_date = str((raw_earnings_string.split('<b>'))[1].split('<font size="-1">')).split("'")[1].strip()
    except:
        return 'Error Parsing'
    
    earnings_time = str(raw_earnings_string[-2:].strip()).strip()
    
    earnings_dict['Date'] = earnings_date
    earnings_dict['Earnings Time'] = earnings_time
    
    # Parsing 7 day implied move if weekly option exists
    ending_string = '<font size="-2">'
    curr_7 = (soup.split(curr7_implied))[1].split(ending_string)[0].replace('\n','').strip("").split("<td>")[-1].strip()
    earnings_dict['Current 7 Day Implied'] = curr_7
    
    # Parsing Weekly Implied move if weekly option exists
    if implied_move_weekly in soup:
        weekly_implied = (soup.split(implied_move_weekly))[1].split(ending_string)[0].replace('\n','').strip("").split("<td>")[-1].strip()
    else:
        weekly_implied = ''
    earnings_dict["Implied Move Weekly"] = weekly_implied
    
    return earnings_dict

# Looping through the soup lxml text table format
# and splitting each row as a individual string
# and parsing string to retrieve the date,
# open, and close information.

def yahoo_table_parse(raw_html_table):
    tickers = []
    call_times = []
    implied_7_day = []
    implied_weekly = []
    eps = []
    i = 1
    end_row = 10
    for row in raw_html_table.find_all('tr'):
        # Individual row stores current row item and delimits on '\n'
        individual_row = str(row).split('\n')
        row_items = individual_row[0].split('</span>')[:3]

        if i == 1:
            i += 1
            continue
        tick = row_items[0].split('data-symbol="')[1].split('"')[0]
        os_check = optionslam_scrape(tick)

        if type(os_check) == str:
            continue
        else:
            tickers.append(tick)
            call_times.append(row_items[0].split('data-symbol="')[1].split('"')[-1].replace('>',''))
            eps.append(row_items[1].split('</td>')[1].split('>')[1])
            implied_7_day.append(os_check['Current 7 Day Implied'].replace('%',''))
            implied_weekly.append(os_check['Implied Move Weekly'].replace('%',''))


    return pd.DataFrame({'Tickers': tickers, 'Call Times': call_times, 'EPS': eps,
                         'Current 7 Day Implied': implied_7_day,
                         'Implied Move Weekly': implied_weekly})


def yahoo_earnings(date):
    # Yahoo Earnings Calendar Check

    today = date.strftime('%Y-%m-%d')
    tables = []
    for i in range(6):
        yahoo_url = 'https://finance.yahoo.com/calendar/earnings?day=' + today + '&offset={}&size=100'.format(int(i*100))
        res = requests.get(yahoo_url)
        soup = bs(requests.get(yahoo_url).text, "lxml")

        try:
            table = soup.find_all('table')[0]
            tables.append(yahoo_table_parse(table))
        except:
            print('No Table')

    return pd.concat(tables,axis = 0, ignore_index = True)


def close_data(ticker_lst, start_date = dt.datetime(2018, 2, 20)):
    # Define which online source one should use
    data_source = 'yahoo'

    end = dt.datetime.today()

    # User pandas_reader.data.DataReader to load the desired data. As simple as that.
    panel_data = datareader.DataReader(ticker_lst, data_source, start_date, end)

    # Getting just the adjusted closing prices. This will return a Pandas DataFrame
    # The index in this DataFrame is the major index of the panel_data.
    return panel_data.ix['Close']

'''
Function for pulling latest SPX, VIX, VVIX, or SKEW data. Input is a string, pulls 
the latest 2 lines of data from yahoo finance for given ticker and returns a 
dataframe of the open and close with the latest date as the first row.
'''
def latest_yahoo(ticker):
    # Using requests to ping yahoo finance to retrieve 
    # historical data table
    site = 'https://finance.yahoo.com/quote/{0}/history?p={0}'.format(ticker)
    
    res = requests.get(site)
    soup = bs(res.text, 'lxml')
    table = soup.find_all('table')[0]

    # Looping through the soup lxml text table format
    # and splitting each row as a individual string
    # and parsing string to retrieve the date,
    # open, and close information.
    i = 1
    end_row = 3
    for row in table.find_all('tr'):
        # Individual row stores current row item and delimits on '\n'
        individual_row = str(row).split('\n')
        
        # row_items is parsed string for each current row where each
        # item in list is the date, open, high, low, close, and volume
        row_items = [item.split('>')[1] for item in [string.split('</span>')[0] for string in individual_row[0].split('<span ')[1:]]]
        
        if i == 1:
            # Skip first row because they are column headers
            i += 1
            continue
        elif i == end_row:
            break
        else:
            # Append necessary items to initialized lists for 
            # dataframe storage
            close = float(row_items[5].replace(',',''))
        i += 1
    
    # Return dataframe of necessary values
    return np.round(np.float(close),2)

def last_close(ticker):
    alphavantage_link = 'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol={0}&apikey=5HZEUI5AFJB06BUK&datatype=csv&outputsize=compact'.format(ticker)
    stockframe = pd.read_csv(alphavantage_link, index_col = 0).sort_index().tail(1)['close']
    
    return stockframe

# Function for calculating standard dev and price moves in terms of standard dev
# DF[[Adj Close]] Rolling Period --> DF[['Daily Vol','Daily Price Vol','Price Dev','Annual Vol']]
def price_devs(ticker, lookbackwindow, rollingperiod):
    # Define which online source one should use
    data_source = 'yahoo'
    
    end = dt.datetime.today()
    start_date = end - dt.timedelta(days = lookbackwindow)
    
    # User pandas_reader.data.DataReader to load the desired data. As simple as that.
    df = datareader.DataReader([ticker], data_source, start_date, end).sort_index()

    # Getting just the adjusted closing prices. This will return a Pandas DataFrame
    # The index in this DataFrame is the major index of the panel_data.
    df = df.ix['Close'].sort_index()
    
    df.columns = ['prices']
    df['prices_delta'] = df.prices - df.prices.shift(1)
    df['log_returns'] = np.log(df.prices) - np.log(df.prices.shift(1))
    df['daily_vol'] = st.rolling_std(df.log_returns, rollingperiod, ddof = 1)
    df['daily_vol_dollar'] = df.daily_vol*df.prices
    df['price_dev'] = df.prices_delta/df.daily_vol_dollar.shift(1)
    df['annual_vol'] = df.daily_vol*np.sqrt(252)
    return df


'''
Function for getting all relevant earnings for a given starting week (Monday in dt.datetime(YYYY, m, d) format)
Returns a dataframe with the earnings names, implied move, price, and earnings times.
'''
def weekly_earnings_check(start_datetime, days_forward):

    start_date = start_datetime

    weekly_earnings = []
    while start_date.weekday() < days_forward:
        temp_earnings = yahoo_earnings(start_date)
        temp_earnings['Earnings Date'] = start_date
        temp_earnings['Last Close'] = 0
        for idx, row in temp_earnings.iterrows():
            temp_earnings.loc[idx, 'Last Close'] = latest_yahoo(row['Tickers'])
        weekly_earnings.append(temp_earnings)
        start_date = start_date + dt.timedelta(1)

    earnings_df = pd.concat(weekly_earnings,axis = 0, ignore_index = True)
    earnings_df = earnings_df[earnings_df['Last Close'] >= 30]
    earnings_df['Implied Move Weekly'] = pd.to_numeric(earnings_df['Implied Move Weekly'])
    #earnings_df = earnings_df[earnings_df['Call Times'] != '-']
    earnings_df['Lower Bound'] = np.round(earnings_df['Last Close']*(1 - earnings_df['Implied Move Weekly']/100),2)
    earnings_df = earnings_df.sort_values(['Earnings Date','Call Times'])
    
    return earnings_df

'''
Function for pulling options data from yahoo Input is a string, either 'call' or 'put' to
determine the contract type to pull from yahoo finance. The output is a dataframe of the latest
data from yahoo finance tagged with the current date-time. Output columns are pull date-time,
contract name, strike, last price, bid, ask volume, open interest, and IV (in decimal form).
'''
def yahoo_options(contract = 'put', ticker = 'SPX'):
    # Using request to ping yahoo and retrieve the raw html
    # tables for calls and puts data for gspc
    if ticker == 'SPX':
        site = 'https://finance.yahoo.com/quote/%5EGSPC/options?p=%5EGSPC'
    else:
        site = 'https://finance.yahoo.com/quote/{0}/options?p={0}&straddle=false'.format(ticker)
    res = requests.get(site)
    soup = bs(res.text, 'lxml')
    calls = soup.find_all('table')[0]
    puts = soup.find_all('table')[1]
    if contract == 'call':
        table = calls
    else:
        table = puts
        
    # parsing calls table
    
    # initiating data lists for storing column data
    dates = []
    names = []
    strikes = []
    lprices = []
    bids = []
    asks = []
    volumes = []
    open_interests = []
    iv = []
    current_time = dt.datetime.now()
    
    # starting with counter i so that the first row in the
    # specified table is noted as the header row
    i = 1
    for row in table.find_all('tr'):
        rowstring = str(row).split('\n')
        # if first row, store data as header labels for dataframe use
        if i == 1:
            header = [data.split('>')[1] for data in \
                        [rawstring.split('</span>')[0] for \
                         rawstring in rowstring[0].split('<span')[1:]]]
        else:
        # parsing other raw row strings to retrieve necessary data and append them
        # to their corresponding list.
            row_data = rowstring[0].split('data-reactid=')[3:]
            dates.append(current_time)
            names.append(row_data[0].split('title="')[1].split('"')[0])
            strikes.append(float(row_data[3].split('</a>')[0].split('>')[1].replace(',','')))
            lprices.append(float(row_data[4].split('</td>')[0].split('>')[1].replace(',','')))
            bids.append(float(row_data[5].split('</td>')[0].split('>')[1].replace(',','')))
            asks.append(float(row_data[6].split('</td>')[0].split('>')[1].replace(',','')))
            volumes.append(float(row_data[-3].split('</td>')[0].split('>')[1].replace(',','')))
            open_interests.append(float(row_data[-2].split('</td>')[0].split('>')[1].replace(',','')))
            iv.append(float(row_data[-1].split('</td>')[0].split('>')[1].replace(',','').replace('%',''))/100)
        i += 1

    return pd.DataFrame({'Pull Date': dates,
                          contract + ' ' + header[0]: names,
                          header[2]: strikes,
                          header[3]: lprices,
                          header[4]: bids,
                          header[5]: asks,
                          header[-3]: volumes,
                          header[-2]: open_interests,
                          header[-1]: iv})

def fundamentals(ticker):

    site = 'https://finance.yahoo.com/quote/{0}?p={0}'.format(ticker)

    res = requests.get(site)
    soup = bs(res.text, 'lxml')
    table = soup.find_all('table')[1]
    sum_dict = {}

    # Looping through the soup lxml text table format
    # and splitting each row as a individual string
    # and parsing string to retrieve the date,
    # open, and close information.


    for row in table.find_all('tr'):
        # Individual row stores current row item and delimits on '\n'
        individual_row = str(row).split('\n')[0]

        # row_items is parsed string for each current row where each
        # item in list is the date, open, high, low, close, and volume
        row_items = individual_row.split('<span data-reactid=')[1].split('"><!-- react-text: ')
        if len(row_items) > 1:
            sum_item = row_items[0].split('>')[1].split('<')[0]
            sum_value = row_items[1].split('-->')[1].split('<')[0]
        elif 'YIELD' in row_items[0]:
            try:
                temp_val = row_items[0].split('-value">')[1].split("</td>")[0]
                div_amount = float(temp_val.split(' ')[0])
                div_yield = float(temp_val.split(' ')[1].replace('(','').replace(')','').replace('%',''))

                sum_dict['Div'] = div_amount
                sum_dict['Yield'] = div_yield
            except:
                sum_dict['Div'] = np.nan
                sum_dict['Yield'] = np.nan

        sum_dict[sum_item] = sum_value

    return pd.DataFrame(sum_dict, index = [ticker])

# Function to return fundametal data of a ticker list
def get_fundas(ticker_lst):
    fund_lst = []
    for tick in ticker_lst:
        fund_lst.append(fundamentals(tick))
    return pd.concat(fund_lst,axis = 0)

# Function to pull straddled view of options of a ticker

def get_option_chain(ticker):
    putframe = yahoo_options(contract = 'put', ticker = ticker)
    callframe = yahoo_options(contract = 'call', ticker = ticker)
    calls = callframe[['Ask','Bid','Implied Volatility','Last Price','Open Interest','Strike','Volume']]
    puts = putframe[['Ask','Bid','Implied Volatility','Last Price','Open Interest','Strike','Volume']]
    return puts.merge(calls, how = 'inner', on = 'Strike', suffixes=('_C', '_P'))


In [18]:
q3_earnings_weeks = [dt.datetime(2018,7,16), dt.datetime(2018,7,23), dt.datetime(2018,7,30)]

may14_earnings = weekly_earnings_check(dt.datetime(2018,5,14), 5)

No Table
No Table
No Table
No Table
No Table
No Table
No Table
No Table
No Table
No Table
No Table
No Table
No Table
No Table
No Table
No Table
No Table
No Table
No Table
No Table
No Table
No Table
No Table
No Table
No Table


In [19]:
may14_earnings

,Call Times,Current 7 Day Implied,EPS,Implied Move Weekly,Tickers,Earnings Date,Last Close,Lower Bound
0,Before Market Open,3.90,0.45,11.17,DKS,2018-05-14,31.77,28.22
3,After Market Close,4.01,0.65,5.01,CSCO,2018-05-16,45.98,43.68
4,After Market Close,2.36,0.46,6.26,CRM,2018-05-16,130.38,122.22
5,After Market Close,4.86,1.14,6.48,AMAT,2018-05-17,54.98,51.42


In [26]:
last_close('GOOG').tail(1)

timestamp
2018-05-11    1098.26
Name: close, dtype: float64